In [1]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import wget

In [2]:
filename = wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

In [ ]:
app_filename = wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py")

In [2]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload_Mass(kg)'].max()
min_payload = spacex_df['Payload_Mass(kg)'].min()

In [3]:
spacex_df.head()

,Unnamed: 0,Flight_Number,Launch_Site,class,Payload_Mass(kg),Booster_Version,Booster_Version_Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [4]:
# Create a dash application
app = dash.Dash(__name__)

In [5]:
# Create an app layout
# TASK 1: Add a dropdown list to enable Launch Site selection
# The default select value is for ALL sites # dcc.Dropdown(id='site-dropdown',...)

In [6]:
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                dcc.Dropdown(
                                    id='site-dropdown',
                                    options=[
                                        {'label': 'All Launch Sites', 'value':'All'},
                                        {'label': 'CCAFS LC-40', 'value':'CCAFS LC-40'},
                                        {'label': 'CCAFS SLC-40', 'value':'CCAFS SLC-40'},
                                        {'label': 'KSC LC-39A', 'value':'KSC LC-39A'},
                                        {'label': 'VAFB SLC-4E', 'value':'VAFB SLC-4E'},
                                        
                                    ],
                                    value = 'All',
                                    placeholder = 'Select Launch Site',
                                    searchable = True
                                    ),
                                    html.Br(),
                                
                                    html.Div(dcc.Graph(id='success-pie-chart')),
                                    html.Br(),
                                
                                    html.P("Payload range (Kg):"),
                                    html.Div(dcc.RangeSlider(id='payload-slider', min=0,max=10000,step=1000, 
                                                             marks={0:'0', 100:'100'},
                                                             tooltip={"placement": "top", "always_visible": True},
                                                            value=[min_payload, max_payload])),
                                    
                                    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                               ])
                                #html.Br(),
                              
                
                           

In [7]:
# TASK 2: Add a pie chart to show the total successful launches count for all sites
# If a specific launch site was selected, show the Success vs. Failed counts for the site
                                    #html.Div(dcc.Graph(id='success-pie-chart')),
                                #html.Br(),

                                #html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                #html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                               # ])

In [8]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output('success-pie-chart', 'figure'),
    Input('site-dropdown','value'))

def generate_pie_chart(entered_launch_site):
    if entered_launch_site == 'All':
        fig =px.pie(spacex_df, values='class', names='Launch_Site', title='Total successful count of all launch sites')
        return fig
    else:
        selected_site_df = spacex_df[spacex_df['Launch_Site'] == entered_launch_site]
        selected_site_df = selected_site_df.groupby(['Launch_Site','class']).size().reset_index(name='count of class')
        fig =px.pie(selected_site_df, names='class', values='count of class', title=f"Total count of launch site: {entered_launch_site}")
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('site-dropdown','value'),
     Input('payload-slider','value')])

def generate_scatter_chart(entered_launch_site, payload_range):
    payload_value_low, payload_value_high = payload_range
    
    if entered_launch_site == 'All':
        fig =px.scatter(spacex_df, x= 'Payload_Mass(kg)', y='class', title = 'Correlation between payload and launch success',
                       color = 'Booster_Version')
        return fig
    else:
        selected_site_df = spacex_df[spacex_df['Launch_Site'] == entered_launch_site]
        payload_range_df = selected_site_df[(selected_site_df['Payload_Mass(kg)'] >= payload_value_low) & (selected_site_df['Payload_Mass(kg)'] <= payload_value_high)]
        fig =px.scatter(payload_range_df, x= 'Payload_Mass(kg)', y='class', title = f'Correlation between payload and launch success for launch site:{entered_launch_site}', 
                       color = 'Booster_Version')
        return fig



In [9]:
# Run the app
if __name__ == '__main__':
    app.run_server()